In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, RandomFlip, RandomRotation, GlobalAveragePooling2D
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from dataGenerator import generateEqualDataSet
from tensorflow.keras.applications.vgg16 import VGG16

from keras.preprocessing.image import ImageDataGenerator

In [34]:
datagen = ImageDataGenerator(
    rotation_range=40,
    horizontal_flip=True
)

In [35]:
baseModel = VGG16(include_top=False, input_shape=(500,500,3))
baseModel.trainable = False

model = Sequential([
    baseModel,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

lr_schedule = ExponentialDecay(
    0.001,
    1000,
    0.99,
    staircase=True
)
optimizer = Adam(learning_rate=lr_schedule)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
baseModel = VGG16(include_top=False, input_shape=(500,500,3))
baseModel.trainable = False

inputs = Input(shape=(500,500, 3))
x = baseModel(inputs, training=False)
x = GlobalAveragePooling2D()(x)

outputs = Dense(1)(x)
model = Model(inputs, outputs)



model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


NameError: name 'GlobalAveragePooling2D' is not defined

In [2]:
model = Sequential();
data_augmentation = Sequential([
  RandomFlip("horizontal_and_vertical"),
  RandomRotation(0.2),
])
model.add(VGG16())
model.add(data_augmentation)
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(500, 500, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', 'precision'])

model.summary()

553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 31s 0us/step


c:\Users\adijo\.conda\envs\swimmingPoolDetector\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ ?                      │   138,357,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 138,357,544 (527.79 MB)

 Trainable params: 138,357,544 (527.79 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
data_augmentation = Sequential([
  RandomFlip("horizontal_and_vertical"),
  RandomRotation(0.2),
])

In [5]:
X, y = generateEqualDataSet(114)
print(X.shape)
print(y.shape)

(230, 500, 500, 3)
(230,)


In [16]:
xTrain, xTest, yTrain, ytest = train_test_split(X, y, test_size=0.2)
callbacks = [
    ModelCheckpoint("vgg16.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),
    EarlyStopping(monitor="val_loss", patience=2),
]

In [17]:
print(xTrain.shape)
print(yTrain.shape)

(184, 500, 500, 3)
(184,)


In [36]:
datagen.fit(xTrain)

In [42]:
checkpoint = ModelCheckpoint("vgg16.h5", monitor='accuracy', verbose=1, save_best_only=True, mode='max')
earlyStopping = EarlyStopping(monitor="loss", patience=2),

callbacks = [checkpoint, earlyStopping]

model.fit(datagen.flow(xTrain, yTrain, batch_size=6),
    steps_per_epoch= len(xTrain) / 6,
    epochs=2,
    callbacks=callbacks,)

Epoch 1/2
31/30 [==============================] - ETA: 0s - loss: 1.4702 - accuracy: 0.7554
Epoch 1: accuracy improved from -inf to 0.75543, saving model to vgg16.h5
30/30 [==============================] - 29s 952ms/step - loss: 1.4702 - accuracy: 0.7554
Epoch 2/2
31/30 [==============================] - ETA: 0s - loss: 0.9036 - accuracy: 0.8098
Epoch 2: accuracy improved from 0.75543 to 0.80978, saving model to vgg16.h5
30/30 [==============================] - 34s 1s/step - loss: 0.9036 - accuracy: 0.8098


In [43]:
score = model.evaluate(xTest, ytest, verbose=0)

In [44]:
score

[1.7327345609664917, 0.717391312122345]

In [40]:
xTest.shape

(9, 500, 500, 3)

In [41]:
model.predict(xTest)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step


array([[0.49780837],
       [0.5005815 ],
       [0.50895715],
       [0.5135175 ],
       [0.50448704],
       [0.5076152 ],
       [0.49349058],
       [0.5105456 ],
       [0.50178707]], dtype=float32)